# Ukraine Energy Dashboard - Data Inspection


In [1]:
# Import necessary libraries
import geopandas as gpd

gdf_osm = gpd.read_file("/workspaces/ukr_energy_dash/assets/data/power_stations_with_oblasts.geojson")

# ignore 'osm_id' and 'osm_type' columns
gdf_osm = gdf_osm.drop(columns=["osm_id", "osm_type"])

# rename columns with ":", e.g. "power:source" to "power_source"
gdf_osm = gdf_osm.rename(columns=lambda x: x.replace(":", "_"))

# calculate centroid and add lat/lon columns
gdf_osm["centroid"] = gdf_osm.geometry.centroid
gdf_osm["centroid_lon"] = gdf_osm.centroid.x
gdf_osm["centroid_lat"] = gdf_osm.centroid.y

/tmp/ipykernel_10178/1338144962.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_osm["centroid"] = gdf_osm.geometry.centroid
/tmp/ipykernel_10178/1338144962.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_osm["centroid_lon"] = gdf_osm.centroid.x
/tmp/ipykernel_10178/1338144962.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_osm["centroid_lat"] = gdf_osm.centroid.y


In [2]:
gdf_osm.shape

(999, 18)

In [3]:
# sort by oblast, then power, power:source
gdf_osm = gdf_osm.sort_values(by=["oblast_name_en", "name", "power", "plant_method"], ascending=[True, True, True, True], ignore_index=True)

In [4]:
gdf_osm.head()

,power,substation,name,station_name_en,operator,operator_en,barrier,voltage,landuse,plant_method,plant_output_electricity,plant_source,oblast_name_en,gppd_overlap,geometry,centroid,centroid_lon,centroid_lat
0,plant,None,Вороненська ГЕС,None,None,None,None,None,industrial,None,yes,hydro,Cherkasy,False,"POLYGON ((30.23471 49.14875, 30.23474 49.14865...",POINT (30.23487 49.14871),30.234865,49.148713
1,plant,None,Закинута гідроелектростанція,None,None,None,None,None,None,None,yes,hydro,Cherkasy,False,"POLYGON ((30.39681 49.09151, 30.39702 49.09146...",POINT (30.39686 49.09137),30.396857,49.091373
2,plant,None,Звенигородська ГЕС,None,"ЗОВНІШНЬОЕКОНОМІЧНА АСОЦІАЦІЯ ""НОВОСВІТ""",None,None,None,industrial,water-storage,0.4 MW,hydro,Cherkasy,False,"POLYGON ((30.94182 49.09441, 30.94192 49.09439...",POINT (30.94264 49.09422),30.942636,49.094220
3,plant,None,Канівська ГЕС,None,None,None,None,None,industrial,run-of-the-river,472 MW,hydro,Cherkasy,True,"POLYGON ((31.45975 49.76252, 31.45949 49.76242...",POINT (31.47335 49.76075),31.473345,49.760754
4,plant,None,Корсунь-Шевченківська ГЕС,None,None,None,None,None,industrial,None,1.8 MW,hydro,Cherkasy,False,"POLYGON ((31.26293 49.41092, 31.26287 49.41129...",POINT (31.26386 49.41185),31.263858,49.411851


In [5]:
# counts per power source
gdf_osm["plant_source"].value_counts()

plant_source
solar             380
gas               136
hydro              87
coal               21
wind               19
nuclear             5
biogas              4
biomass             3
gas;oil             3
gas;coal            2
diesel              1
coal;gas;oil        1
coal;oil            1
waste               1
oil;gas             1
oil;gas;diesel      1
gas;mazut           1
wood                1
coal;gas            1
Name: count, dtype: int64

In [6]:
gdf_osm.columns

Index(['power', 'substation', 'name', 'station_name_en', 'operator',
       'operator_en', 'barrier', 'voltage', 'landuse', 'plant_method',
       'plant_output_electricity', 'plant_source', 'oblast_name_en',
       'gppd_overlap', 'geometry', 'centroid', 'centroid_lon', 'centroid_lat'],
      dtype='object')

In [7]:
# show all nuclear power plants
gdf_osm[gdf_osm["plant_source"] == "nuclear"]

,power,substation,name,station_name_en,operator,operator_en,barrier,voltage,landuse,plant_method,plant_output_electricity,plant_source,oblast_name_en,gppd_overlap,geometry,centroid,centroid_lon,centroid_lat
404,plant,None,Хмельницька атомна електростанція,Khmelnytskyi Nuclear Power Plant,None,None,None,None,industrial,fission,2000 MW,nuclear,Khmelnytskyi,True,"POLYGON ((26.64097 50.30253, 26.64126 50.30301...",POINT (26.64802 50.30187),26.648019,50.301869
505,plant,None,Чорнобильська Атомна Електростанція,Chornobyl Nuclear Power Plant,НАЕК Енергоатом,None,None,None,industrial,fission,0,nuclear,Kiev,False,"POLYGON ((30.11168 51.38642, 30.11181 51.38643...",POINT (30.10422 51.38981),30.104223,51.389813
675,plant,None,Південноукраїнська АЕС,South Ukraine Nuclear Power Plant,НАЕК Енергоатом,None,None,None,industrial,fission,3000 MW,nuclear,Mykolayiv,True,"MULTIPOLYGON (((31.21935 47.8105, 31.21935 47....",POINT (31.21744 47.81041),31.217440,47.810405
790,plant,None,Рівненська АЕС,Rivne Nuclear Power Plant,НАЕК Енергоатом,None,None,None,None,fission,2.835 GW,nuclear,Rivne,True,"POLYGON ((25.8965 51.31777, 25.89237 51.31929,...",POINT (25.8964 51.32558),25.896396,51.325577
954,plant,None,Запорізька АЕС,Zaporizhzhia Nuclear Power Plant,"НАЕК ""Енергоатом""",None,None,None,industrial,fission,5700 MW,nuclear,Zaporizhia,True,"POLYGON ((34.59054 47.50359, 34.58451 47.505, ...",POINT (34.58812 47.51016),34.588120,47.510159


In [8]:
# sanity check filter for "Rivne Nuclear Power Plant"
gdf_osm[gdf_osm["station_name_en"] == "Rivne Nuclear Power Plant"]

,power,substation,name,station_name_en,operator,operator_en,barrier,voltage,landuse,plant_method,plant_output_electricity,plant_source,oblast_name_en,gppd_overlap,geometry,centroid,centroid_lon,centroid_lat
790,plant,None,Рівненська АЕС,Rivne Nuclear Power Plant,НАЕК Енергоатом,None,None,None,None,fission,2.835 GW,nuclear,Rivne,True,"POLYGON ((25.8965 51.31777, 25.89237 51.31929,...",POINT (25.8964 51.32558),25.896396,51.325577


In [9]:
# sanity check filter for "Dniester Pumped Storage Power Station"
gdf_osm[gdf_osm["station_name_en"] == "Dniester Pumped Storage Power Station"]

,power,substation,name,station_name_en,operator,operator_en,barrier,voltage,landuse,plant_method,plant_output_electricity,plant_source,oblast_name_en,gppd_overlap,geometry,centroid,centroid_lon,centroid_lat
28,plant,None,Дністровська ГАЕС,Dniester Pumped Storage Power Station,None,None,None,None,None,None,972 MW,hydro,Chernivtsi,True,"MULTIPOLYGON (((27.47952 48.51917, 27.47873 48...",POINT (27.47318 48.51523),27.473183,48.51523


In [10]:
# sanity check Kakhovka hydroelectric dam
gdf_osm[gdf_osm["plant_output_electricity"] == "351 MW"]

,power,substation,name,station_name_en,operator,operator_en,barrier,voltage,landuse,plant_method,plant_output_electricity,plant_source,oblast_name_en,gppd_overlap,geometry,centroid,centroid_lon,centroid_lat
326,plant,None,колишня Каховська ГЕС імені П. С. Непорожнього,ex Kakhovskaya HPP named after P. S. Neporozhny,ВАТ «Укргідроенерго»,None,None,None,industrial,None,351 MW,hydro,Kherson,True,"MULTIPOLYGON (((33.36813 46.78048, 33.37141 46...",POINT (33.36182 46.78296),33.361821,46.782962


### Global Power Plant Database (World Resources Institute)

In [11]:
import pandas as pd
url = "https://raw.githubusercontent.com/wri/global-power-plant-database/master/output_database/global_power_plant_database.csv"

df_gppd = pd.read_csv(url, low_memory=False)

In [12]:
# filter for Ukraine only
df_gppd_ukr = df_gppd[df_gppd["country_long"] == "Ukraine"]
df_gppd_ukr.shape

# select relevant columns
df_gppd_ukr = df_gppd_ukr[['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2019', 'generation_data_source']]

In [13]:
# export both (gdf_osm and df_gppd_ukr) as one Excel file with two sheets
with pd.ExcelWriter("/workspaces/ukr_energy_dash/assets/data/ukr_power_substations.xlsx", engine="openpyxl") as writer:
    gdf_osm.to_excel(writer, sheet_name="OSM Power Stations", index=False)
    df_gppd_ukr.to_excel(writer, sheet_name="GPPD Power Plants", index=False)